# Load Libraries

In [1]:
import sys, os, pygmt, importlib, re, base64, glob
import numpy              as np
import pandas             as pd
import xarray             as xr
import xesmf              as xe
import netCDF4            as nc
import matplotlib.pyplot  as plt
import matplotlib.dates   as mdates
from matplotlib.offsetbox import AnchoredOffsetbox, VPacker, TextArea
from datetime             import timedelta, date, datetime
from pathlib              import Path
from dask.distributed     import Client, LocalCluster
from collections          import defaultdict
from scipy.interpolate    import interp1d
from pyproj               import CRS, Transformer
from IPython.display      import Image, HTML, Video
mod_path = '/home/581/da1339/AFIM/src/AFIM/src'
sys.path.insert(0, mod_path)
from sea_ice_toolbox      import SeaIceToolbox, SeaIceToolboxManager

## reload SeaIceToolbox if local changes have been made

In [ ]:
import sys
import importlib
for mod in list(sys.modules):
    if mod.startswith("sea_ice_toolbox") or mod.startswith("sea_ice_"):
        del sys.modules[mod]
import sea_ice_plotter
import sea_ice_classification
import sea_ice_icebergs
import sea_ice_observations
import sea_ice_metrics
import sea_ice_toolbox
importlib.reload(sea_ice_plotter)
importlib.reload(sea_ice_classification)
importlib.reload(sea_ice_icebergs)
importlib.reload(sea_ice_observations)
importlib.reload(sea_ice_metrics)
importlib.reload(sea_ice_toolbox)
from sea_ice_toolbox import SeaIceToolbox

# [Simulation Table](https://dpath2o.github.io/AFIM/ice_diag_summary.html)

# [Methodology](https://dpath2o.github.io/AFIM/AFIM_sensitivity_methodology.html)

# ACCESS-OM2-025-ERA5 sea ice data ( **needs only to be run once, after which data can be extracted from local zarr files** )
+ this uses [ACCESS-NRI intake catalog](https://access-nri-intake-catalog.readthedocs.io/en/latest/index.html) through a ``SeaIceToolbox`` method ``load_ACCESS_OM_CICE``

In [ ]:
dt0_str        = "2000-01-01"
dtN_str        = "2023-12-31"
dt_rng_str     = f"{dt0_str[:4]}-{dtN_str[:4]}"
ice_type       = "FI_BT"
SIA_dict       = {}
FIA_dict       = {}
SI_tools       = SeaIceToolbox(sim_name             = "AOM2-ERA5",
                               dt0_str              = dt0_str,
                               dtN_str              = dtN_str,
                               ice_speed_threshold  = 5e-4,
                               ice_speed_type       = "BT",
                               ice_type             = "FI_BT",
                               overwrite_zarr       = False,
                               save_new_figs        = True,
                               show_figs            = True,
                               overwrite_saved_figs = True)

## load ACCESS-OM-025-ERA5 data from ESM datastore then convert to zarr files for easy of use later
+ loading and saving only needs to be done if it has not been done previously
+ running both of these cells takes approximately an hour with the bottleneck at writing to zarr

In [ ]:
AOM2 = SI_tools.load_ACCESS_OM_CICE()

In [ ]:
SI_tools.write_ACCESS_to_monthly_zarr(AOM2)

## run ACCESS-OM-025-ERA5 through fast ice classification workflow
+ this is a quick process and will save masked zarr dataset

In [ ]:
AOM2_FI_raw = SI_tools.process_daily_cice(ispd_thresh=5e-4,
                                          ivec_type="BT",
                                          overwrite_zarr_group=True)

In [ ]:
AOM2_FI_roll = SI_tools.process_rolling_cice(ispd_thresh=5e-4,
                                             ivec_type="BT",
                                             overwrite_zarr_group=True)

# <span style="color: red;">Study Objectives</span>

# Can a realistic simulation of circumpolar Antarctic fast ice be achieved in CICE-standalone?

+ Sea ice area and volum comparisons are done against [NSIDC](https://nsidc.org/data/g02202/versions/4) and [ACCESS-OM2-025-IAF-ERA](https://forum.access-hive.org.au/t/era-5-forced-access-om2-simulations/1103) (``AOM2-ERA``). The rationale is to show two of my simulations ([AFIM](https://github.com/dpath2o/AFIM), *CICE6-standalone*) against both NSIDC and ACCESS-OM2 as a relative overall gauge of AFIM performance.

+ I've chosen two related AFIM simulations: ``elps-min`` and ``gi-nil``. Both of which have ellispse eccentrincities ($e_{f}=e_{g}=1.2$, and further just abbreviated $e$) and tensile stress parameter ($k_{t}=0.2$) and all other CICE namelist parameters set to *default* standalone configuration, with the exception of the landmask file used (``kmt_file``). ``elps-min`` uses a modified landmask where 25% of the non-isolated grounded iceberg cells remain, whereas ``gi-nil`` uses the same landmask file as ``AOM2-ERA5`` and hence has no grounded iceberg cells.
  
+ Animations of daily sea ice speed ($\sqrt{u^2 + v^2}$) are then shown of the above two AFIM simulations along with AOM2-ERA5 for the same period (Austral winter 1999). Implicitly, the animations show the effect of $e$ and $ktens$ on the coastal icepack, and for the ``elps-min`` simulation the animation clearly shows the significance of grouned icebergs. 

## Southern Ocean Pack/Fast Ice Area, Volume, Thickness and Drift/Speed Comparisons
+ run the script:
```bash qsub ./scripts/all_stats.pbs```

In [ ]:
def load_ispd_diffs(sim_name):
    D_search = Path(Path.home(), "seaice", "OSI_SAF", "ice_drift_455m")
    F_search = "ispd_diffs_pygmt_nn_{sim}_199*.nc".format(sim=sim_name)
    P_       = sorted(D_search.rglob(F_search))
    return xr.open_mfdataset(P_, combine="by_coords")
ISP_bias             = {}
ISP_rmse             = {}
ISP_ang              = {}
ISP_cos              = {}
ds                   = load_ispd_diffs("elps-min")
ISP_bias['elps-min'] = {'ispd_bias'  : ds['d_ispd_CICE'].mean(dim=['ny','nx']).compute()}#.groupby('time.dayofyear').mean('time').mean(dim=['ny','nx']).compute(),
ISP_rmse['elps-min'] = {'ispd_rmse'  : ds['RMSE_CICE']}#.groupby('time.dayofyear').mean('time').compute(),
ISP_ang['elps-min']  = {'ang_bias'   : ds['ANG_CICE_mean']}#.groupby('time.dayofyear').mean('time').compute(),
ISP_cos['elps-min']  = {'cos_bias'   : ds['COS_CICE_mean']}#.groupby('time.dayofyear').mean('time').compute(),
                         #'line_color' : 'red'}
ISP_bias['AOM2-ERA5'] = {'ispd_bias'  : ds['d_ispd_AOM2'].mean(dim=['ny','nx']).compute()}#.groupby('time.dayofyear').mean('time').mean(dim=['ny','nx']).compute(),
ISP_rmse['AOM2-ERA5'] = {'ispd_rmse'  : ds['RMSE_AOM2']}#.groupby('time.dayofyear').mean('time').compute(),
ISP_ang['AOM2-ERA5']  = {'ang_bias'   : ds['ANG_AOM2_mean']}#.groupby('time.dayofyear').mean('time').compute(),
ISP_cos['AOM2-ERA5']  = {'cos_bias'   : ds['COS_AOM2_mean']}#.groupby('time.dayofyear').mean('time').compute(),
                         #'line_color' : 'green'}

In [ ]:
ISP_bias['ORAS'] = {'ispd_bias'  : ds['d_ispd_ORAS'].mean(dim=['ny','nx']).compute()}#.groupby('time.dayofyear').mean('time').mean(dim=['ny','nx']).compute(),
ISP_rmse['ORAS'] = {'ispd_rmse'  : ds['RMSE_ORAS']}#.groupby('time.dayofyear').mean('time').compute(),
ISP_ang['ORAS']  = {'ang_bias'   : ds['ANG_ORAS_mean']}#.groupby('time.dayofyear').mean('time').compute(),
ISP_cos['ORAS']  = {'cos_bias'   : ds['COS_ORAS_mean']}#.groupby('time.dayofyear').mean('time').compute(),
                         #'line_color' : 'orange'}
ds                     = load_ispd_diffs("notensnogi")
ISP_bias['notensnogi'] = {'ispd_bias'  : ds['d_ispd_CICE'].mean(dim=['ny','nx']).compute()}#.groupby('time.dayofyear').mean('time').mean(dim=['ny','nx']).compute(),
ISP_rmse['notensnogi'] = {'ispd_rmse'  : ds['RMSE_CICE']}#.groupby('time.dayofyear').mean('time').compute(),
ISP_ang['notensnogi']  = {'ang_bias'   : ds['ANG_CICE_mean']}#.groupby('time.dayofyear').mean('time').compute(),
ISP_cos['notensnogi']  = {'cos_bias'   : ds['COS_CICE_mean']}#.groupby('time.dayofyear').mean('time').compute(),
                         #'line_color' : 'pink'}

In [ ]:
sim_name    = "elps-min"
dt0_str     = "1994-01-01"
dtN_str     = "1999-12-31"
P_log       = Path(Path.home(), "logs", "paper1.log")
SI_tool_mgr = SeaIceToolboxManager(P_log=P_log)
sim_tools   = SI_tool_mgr.get_toolbox(sim_name, dt0_str=dt0_str, dtN_str=dtN_str)
sim_tools.pygmt_timeseries(ISP_cos,
                        comp_name    = "ISP_cos_bias",
                        primary_key  = "cos_bias",
                        climatology  = True,
                        ylabel       = "Sea Ice Drift Cosine Simality",
                        ylim         = [-.6,.6],
                        ytick_pri    = .1,
                        ytick_sec    = .05,
                           legend_pos = "JBC+jBC+o0.2c+w5c",
                          show_fig   = True)

In [ ]:
Image(filename='/g/data/gv90/da1339/GRAPHICAL/AFIM/timeseries/SIV_elps-min_gi-nil_AOM2-ERA5_1994-1999.png')

<a id="spatial-comparisons"></a>
### Spatial Comaprisons 

#### Hemisphere comparisons of sea ice concentration and sea ice thickness

In [ ]:
HTML("""
<video width="1000" controls>
  <source src="https://raw.githubusercontent.com/dpath2o/AFIM/main/docs/figures/elps-min_ispd_BT_Aus_1999-winter.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

##### ``AOM2-ERA5``: e = 2, ktens = None, gi-min = None

In [ ]:
HTML("""
<video width="1000" controls>
  <source src="https://raw.githubusercontent.com/dpath2o/AFIM/main/docs/figures/AOM2-ERA5_ispd_BT_Aus_1999-winter.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

##### ``gi-nil``: e = 2, ktens = 0.2, gi-min = None

In [ ]:
HTML("""
<video width="1000" controls>
  <source src="https://raw.githubusercontent.com/dpath2o/AFIM/main/docs/figures/gi-nil_ispd_BT_Aus_1999-winter.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

## Which is the best method for computing fast ice: binary-day or rolling-mean?

+ create a dictionary of circumpolar fast ice area time series for one simulation (``elps-min``) for a range of different ``binary-day`` and ``rolling-mean`` configurations
+ calculate relevant statistical skills for each configuration against fast ice area observations (``AF2020``)
+ come up with a metric (normalisation-score) for determining the best performing configuration

In [ ]:
FIA_dict   = {}
FIV_dict   = {}
# P_dict     = {}
# divu_dict  = {}
# dvidtt_dict= {}
sim_name   = "elps-min"
dt0_str    = "1994-01-01"
dtN_str    = "1999-12-31"
SI_tools   = SeaIceToolbox(sim_name             = sim_name,
                           dt0_str              = dt0_str,
                           dtN_str              = dtN_str)

In [ ]:
FI_raw, CICE = SI_tools.load_processed_cice( zarr_CICE = True )

#### compute binary-days 
+ this will be binary-days for 7 out of 7, 6 out of 7, 5 out of 7, 8 out of 8, 7 out of 8, 6 out of 8, and so on, until 13 out of 15.
  

In [ ]:
CICE_SO = CICE.isel(nj=SI_tools.hemisphere_dict['nj_slice'])
for win in np.arange(7, 16):  # window sizes: 7 to 15 inclusive
    for cnt in np.arange(win, win - 3, -1):  # max, max-1, max-2 (e.g., 7,6,5)
        key_name = f"elps-min_bin-day_{cnt}of{win}"
        FI_bool               = SI_tools.boolean_fast_ice(FI_raw['FI_mask'], window=win, min_count=cnt)
        aice_bool             = CICE_SO['aice'].where(FI_bool)
        hi_bool               = CICE_SO['hi'].where(FI_bool)
        tarea_bool            = CICE_SO['tarea'].where(FI_bool)
        #P_bool                = CICE_SO['strength'].where(FI_bool)
        #divu_bool             = CICE_SO['divu'].where(FI_bool)
        #dvidtt_bool           = CICE_SO['dvidtt'].where(FI_bool)
        FIA_dict[key_name]    = SI_tools.compute_ice_area(FI_bool, tarea_bool)
        FIV_dict[key_name]    = SI_tools.compute_ice_volume(FI_bool, hi_bool, tarea_bool)
        #P_dict[key_name]      = P_bool.sum(dim=('nj','ni'))
        #divu_dict[key_name]   = divu_bool.sum(dim=('nj','ni'))
        #dvidtt_dict[key_name] = dvidtt_bool.sum(dim=('nj','ni'))

#### compute rolling-mean

+ this will be rolling-means over 7, 10, 13, 16 and 19 day periods
+ then compute FIA on each of those

In [ ]:
for i in np.arange(7,20,3):
    roll_name  = f"elps-min_roll-day_{i:d}"
    CICE_rolls = SI_tools.process_rolling_cice(sim_name         = sim_name,
                                               dt0_str          = dt0_str,
                                               dtN_str          = dtN_str,
                                               mean_period      = i,
                                               ivec_type        = "BT",
                                               write_zarr_group = False)
    FIA_dict[roll_name]    = SI_tools.compute_ice_area(CICE_rolls['FI_mask'], CICE_rolls['tarea'])
    FIV_dict[roll_name]    = SI_tools.compute_ice_volume(CICE_rolls['FI_mask'], CICE_rolls['hi'], CICE_rolls['tarea'])
    #P_FI_roll              = CICE_rolls['strength'].where(CICE_rolls['FI_mask'])
    #divu_FI_roll           = CICE_rolls['divu'].where(CICE_rolls['FI_mask'])
    #dvidtt_FI_roll         = CICE_rolls['dvidtt'].where(CICE_rolls['FI_mask'])
    #P_dict[roll_name]      = P_FI_roll.sum(dim=('nj','ni'))
    #divu_dict[roll_name]   = divu_FI_roll.sum(dim=('nj','ni'))
    #dvidtt_dict[roll_name] = dvidtt_FI_roll.sum(dim=('nj','ni'))

#### compute statistics for each configuration

In [ ]:
# ts_ds     = SI_tools.dict_to_ds(FIA_dict)
# ts_ds_cnk = ts_ds.chunk({'time':30})
# ts_ds_cnk.to_zarr(Path(SI_tools.D_sim,"FIA_time-series_multi-bin-days_multi-roll-days.zarr"), mode="w")
# ts_ds     = SI_tools.dict_to_ds(FIV_dict)
# ts_ds_cnk = ts_ds.chunk({'time':30})
# ts_ds_cnk.to_zarr(Path(SI_tools.D_sim,"FIV_time-series_multi-bin-days_multi-roll-days.zarr"), mode="w")
# ts_ds     = SI_tools.dict_to_ds(P_dict)
# ts_ds_cnk = ts_ds.chunk({'time':30})
# ts_ds_cnk.to_zarr(Path(SI_tools.D_sim,"FI-strength_time-series_multi-bin-days_multi-roll-days.zarr"), mode="w")
# ts_ds     = SI_tools.dict_to_ds(divu_dict)
# ts_ds_cnk = ts_ds.chunk({'time':30})
# ts_ds_cnk.to_zarr(Path(SI_tools.D_sim,"FI-strain-rate_time-series_multi-bin-days_multi-roll-days.zarr"), mode="w")
# ts_ds     = SI_tools.dict_to_ds(dvidtt_dict)
# ts_ds_cnk = ts_ds.chunk({'time':30})
# ts_ds_cnk.to_zarr(Path(SI_tools.D_sim,"FI-thermo-vol-tend_time-series_multi-bin-days_multi-roll-days.zarr"), mode="w")
AF_clim = SI_tools.load_AF2020_FIA_summary(start="1994-01-01", end="1999-12-31")
obs_fia = SI_tools.AF2020_clim_to_model_time( FIA_dict['elps-min_bin-day_7of7'] , AF_clim["FIA_clim"].sel(region="circumpolar"))
FIA_stats = {}
for key in FIA_dict.keys():
    print(key)
    FIA_stats[key] = SI_tools.compute_skill_statistics( FIA_dict[key], obs_fia )

#### compute a normalised-score to rank the configurations

In [ ]:
df['Abs_Bias'] = np.abs(df['Bias'])
df['SD_Diff']  = np.abs(df['SD_Model'] - df['SD_Obs'])
df['score']    = (df['Abs_Bias'] / df['Abs_Bias'].max() +
                   df['RMSE']     / df['RMSE'].max() +
                   df['MAE']      / df['MAE'].max() +
                   df['SD_Diff']  / df['SD_Diff'].max() +
                   (1 - df['Corr']) / (1 - df['Corr'].min()))
best_method    = df['score'].idxmin()
df_sorted      = df.sort_values(by='score')
print("Best-aligned method with observed FIA:", best_method)
df_sorted[['Bias', 'RMSE', 'MAE', 'Corr', 'SD_Model', 'SD_Obs']].head(5)

#### Taylor Diagram

In [ ]:
def plot_taylor_nonlinear_zoom(stats_dict, color_metric="Bias", corr_lims=(0.4, 0.1), std_lims=(0.75, 1.0)):
    def nonlinear_theta(corr, exponent=0.5):
        corr = np.clip(corr, 0, 1)
        linear_angle = np.arccos(corr)
        return np.power(linear_angle / np.pi, exponent) * np.pi
    fig = plt.figure(figsize=(14, 14))
    ax = fig.add_subplot(111, polar=True)
    ax.set_theta_zero_location("E")
    ax.set_theta_direction(-1)
    # Colormap with centered zero
    color_vals = np.array([v[color_metric] for v in stats_dict.values()])
    vmin, vmax = color_vals.min(), color_vals.max()
    norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
    cmap = cm.RdBu_r
    # Set zoomed radial and angular limits
    ax.set_rmin(std_lims[0])
    ax.set_rmax(std_lims[1])
    ax.set_rticks(np.round(np.linspace(*std_lims, 3), 2))
    ax.set_rlabel_position(135)
    corr_min, corr_max = corr_lims
    tick_corrs = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    tick_angles = [np.degrees(nonlinear_theta(c)) for c in tick_corrs]
    ax.set_thetamin(min(tick_angles))
    ax.set_thetamax(max(tick_angles))
    ax.set_thetagrids(tick_angles, labels=[f"{c:.2f}" for c in tick_corrs])
    # Reference std circle at r = 1
    ref_theta = np.linspace(0, np.pi, 500)
    ax.plot(nonlinear_theta(np.cos(ref_theta)), np.ones_like(ref_theta), 'k-', lw=1.2)
    # RMSD arcs
    ref_std = list(stats_dict.values())[0]['SD_Obs']
    rs = np.linspace(std_lims[0], std_lims[1], 400)
    rms_levels = [20, 50, 100]
    for rms in rms_levels:
        rms_norm = rms / ref_std
        theta = np.arccos(np.clip((1 + rs**2 - rms_norm**2) / (2 * rs), -1, 1))
        scaled_theta = nonlinear_theta(np.cos(theta))
        valid = (scaled_theta >= np.radians(ax.get_thetamin())) & (scaled_theta <= np.radians(ax.get_thetamax()))
        ax.plot(scaled_theta[valid], rs[valid], '--', color='lightgray', lw=0.8)
        if np.any(valid):
            label_theta = scaled_theta[valid][5]
            label_r = rs[valid][5] + 0.01
            ax.text(label_theta, label_r, f'RMSE={rms}', fontsize=7, color='gray')

    # Plot data points with external annotations and arrows
    for i, (label, stat) in enumerate(stats_dict.items()):
        corr = np.clip(stat["Corr"], 0, 1)
        std_ratio = stat["SD_Model"] / stat["SD_Obs"]
        angle = nonlinear_theta(corr)
        r = std_ratio
        color = cmap(norm(stat[color_metric]))
        ax.plot(angle, r, 'o', markersize=6, color=color)
        # Offset annotation radially outward
        r_annot = r + 0.05
        ax.annotate(label,
            xy=(angle, r),
            xytext=(angle*0.9, r_annot),
            textcoords='data',
            ha='center', va='center',
            fontsize=8,
            arrowprops=dict(arrowstyle="->", color="gray", lw=0.7)
        )

    # Colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, pad=0.2, orientation="horizontal", shrink=0.75)
    cbar.set_label(f"{color_metric} (m/s)")
    
    plt.tight_layout()
    plt.show()

In [ ]:
sim_name   = "elps-min"
SI_tools   = SeaIceToolbox( sim_name = sim_name )
AF2020_CSV = SI_tools.load_AF2020_FIA_summary()
obs_FIA    = AF2020_CSV['FIA_clim_repeat'].sel(region='circumpolar')
FIA_sim    = xr.open_zarr(Path(Path.home(),"AFIM_archive","elps-min","FIA_time-series_multi-bin-days_multi-roll-days.zarr"))
dt         = pd.to_datetime(FIA_sim["time"].values)
valid_dt   = dt.year > 1993 
FIA_sim    = FIA_sim.sel(time=valid_dt)
print(FIA_sim.time.values)

In [ ]:
AF_clim = SI_tools.load_AF2020_FIA_summary(start="1994-01-01", end="1999-12-31")
obs_fia = SI_tools.AF2020_clim_to_model_time( FIA_sim['elps-min_bin-day_7of7'] , AF_clim["FIA_clim"].sel(region="circumpolar"))
FIA_stats = {}
for key in FIA_sim.keys():
    print(key)
    FIA_stats[key] = SI_tools.compute_skill_statistics( FIA_sim[key], obs_fia )

In [ ]:
# import json
# with open("/g/data/gv90/da1339/afim_output/elps-min/FIA_skill_statistics_1994-1999.json", "w") as f:
#     json.dump(FIA_stats, f, indent=2)
#plot_taylor(FIA_stats)
plot_taylor_nonlinear_zoom(FIA_stats,color_metric="Bias",std_lims=(0.8, 1))

In [ ]:
FIA_dict = {"AF2020"                  : obs_FIA,
            "elps-min_bin-day_13of15"  : FIA_sim['elps-min_bin-day_13of15'],
            "elps-min_bin-day_14of15" : FIA_sim['elps-min_bin-day_14of15'],
            "elps-min_bin-day_15of15" : FIA_sim['elps-min_bin-day_15of15'],
            "elps-min_roll-day_16"    : FIA_sim['elps-min_roll-day_16']}
SI_tools.plot_timeseries_matplotlib(FIA_dict,
                        ylabel    = "Fast Ice Area (1000-km²)",
                        P_png     = Path(SI_tools.D_graph,"timeseries","FIA_elps-min_bin-day_comparison_15day.png"))
SI_tools.plot_monthly_ice_metric_by_year(FIA_dict,
                                        ice_type         = "FIA",
                                        figsize          = (18, 10),
                                        P_png            = Path(SI_tools.D_graph,sim_name,"FIA_elps-min_bin-day_comparison_15day_by_doy.png"),
                                        plot_annotations = False)

In [ ]:
Image(filename="/g/data/gv90/da1339/GRAPHICAL/AFIM/timeseries/FIA_elps-min_bin-day_comparison_7day.png")

In [ ]:
Image(filename="/g/data/gv90/da1339/GRAPHICAL/AFIM/timeseries/FIA_elps-min_bin-day_comparison_11day.png")

In [ ]:
Image(filename="/g/data/gv90/da1339/GRAPHICAL/AFIM/timeseries/FIA_elps-min_bin-day_comparison_15day.png")

In [ ]:
#Image(filename="/g/data/gv90/da1339/GRAPHICAL/AFIM/elps-min/FIA_elps-min_and_AF2020_2000-2018.png")
Image(filename="/g/data/gv90/da1339/GRAPHICAL/AFIM/elps-min/FIA_elps-min_bin-day_comparison_7day_by_doy.png")

In [ ]:
Image(filename="/g/data/gv90/da1339/GRAPHICAL/AFIM/elps-min/FIA_elps-min_bin-day_comparison_11day_by_doy.png")

In [ ]:
Image(filename="/g/data/gv90/da1339/GRAPHICAL/AFIM/elps-min/FIA_elps-min_bin-day_comparison_15day_by_doy.png")

## Can we realistically simulate the:

1.  fast ice area min

2.  fast ice area max

3.  seasonality

4.  thickness

5.  inter-annual variability

6.  all the while maintaining [pack ice (see above)](#spatial-comparisons)

### Fast ice comparison for simulation ``elps-min`` (and maybe ``elps-mid``)

+ ($e=1.2$, $k_t = 0.2$, $GI_{thin}=0.25$) versus [AF2020 FI db](https://tc.copernicus.org/articles/15/5061/2021/)
+ these results are based on 2000 to 2018 direct comparison
  

#### ``elps-min`` compute FIA and plot both continuous time series and climatology (groupby DOY)

In [2]:
sim_name    = "elps-min"
dt0_str     = "2000-01-01"
dtN_str     = "2018-12-31"
P_log       = Path(Path.home(), "logs", "paper1_sandbox.log")
SI_tool_mgr = SeaIceToolboxManager(P_log=P_log)
SI_tools    = SI_tool_mgr.get_toolbox(sim_name = sim_name,
                                      dt0_str  = dt0_str,
                                      dtN_str  = dtN_str)
FI_bin      = SI_tools.load_classified_ice(bin_days=True)['FI_mask']
CICE_SO     = SI_tools.load_cice_zarr(slice_hem=True, variables=['aice','tarea'])
A_SO        = CICE_SO['tarea'].isel(time=0)
FI_binly    = CICE_SO.where(FI_bin)
FI_aice     = FI_binly['aice'].coarsen(time=15,boundary='trim').mean()
FI_mask_mod = xr.where(FI_aice>0,1,0)
D_obs       = Path(SI_tools.AF_FI_dict['D_AF2020_db_org'])
P_orgs      = sorted(D_obs.glob("FastIce_70_*.nc"))
FI_obs      = xr.open_mfdataset(P_orgs, engine='netcdf4', combine='by_coords')
FI_mask     = xr.where(FI_obs['Fast_Ice_Time_series'] >= 4, 1.0, 0.0)
lat_c       = FI_obs.latitude.isel(time=0)
lon_c       = FI_obs.longitude.isel(time=0)
FI_mod_reT  = FI_mask_mod.reindex(time=FI_obs.time, method="nearest")

2025-08-13 12:34:06,515 - INFO - log file connected: /home/581/da1339/logs/paper1_sandbox.log
2025-08-13 12:34:06,519 - INFO - Dask Client Connected
  Dashboard      : /proxy/8787/status
  Threads        : 2
  Threads/Worker : [1, 1]
  Total Memory   : 14.00 GB

2025-08-13 12:34:06,520 - INFO - hemisphere initialised: SH
2025-08-13 12:34:06,520 - INFO - reading /g/data/gv90/da1339/afim_output/elps-min/ice_diag.d to construct /g/data/gv90/da1339/afim_output/elps-min/ice_in_AFIM_subset_elps-min.json
2025-08-13 12:34:06,524 - INFO -  self.ice_class defined as FI_BT
2025-08-13 12:34:06,524 - INFO - --- SeaIceToolbox Summary ---
2025-08-13 12:34:06,525 - INFO - Simulation Name     : elps-min
2025-08-13 12:34:06,525 - INFO - Analysis Start Date : 2000-01-01
2025-08-13 12:34:06,526 - INFO - Analysis End Date   : 2018-12-31
2025-08-13 12:34:06,526 - INFO - Speed Threshold     : 5.0e-04 m/s
2025-08-13 12:34:06,527 - INFO - Speed Type(s)       : BT
2025-08-13 12:34:06,528 - INFO - Ice Type(s)   

In [ ]:
current_year = None
yearly_slices = []
P_zarr = Path(f"/g/data/gv90/da1339/afim_output/elps-min/zarr/FI_diff.zarr")
for i in range(FI_obs.dims['time']):
    ti = pd.to_datetime(FI_mask.isel(time=i).time.values)
    year = ti.year
    P_yr = Path(P_zarr,f"{year}")
    if P_yr.exists():
        print(f"Group '{year}' exists in {P_zarr}")
        continue
    else:
        print(f"Group '{year}' not found in {P_zarr}")
    print(f"{datetime.now()} FI_obs time {ti}")
    FI_obs_reG = SI_tools.pygmt_regrid(FI_mask.isel(time=i), lon_c, lat_c, grid_res="0.1/0.1", region=[-180, 180, -90, 0], search_radius="5k").astype('float32')
    FI_mod_reG = SI_tools.pygmt_regrid(FI_mod_reT.isel(time=i), FI_binly['TLON'], FI_binly['TLAT'], grid_res="0.1/0.1", region=[-180, 180, -90, 0], search_radius="20k").astype('float32')
    both_valid = xr.where(xr.ufuncs.isfinite(FI_obs_reG) & xr.ufuncs.isfinite(FI_mod_reG), 1, np.nan)
    diff_i = (FI_mod_reG - FI_obs_reG) * both_valid
    diff_i = diff_i.assign_coords(time=ti)
    if current_year is None:
        current_year = year
    if year != current_year:
        # Concatenate and write the previous year's data
        ds_year = xr.concat(yearly_slices, dim="time").to_dataset(name="FI_diff")
        P_zarr.parent.mkdir(parents=True, exist_ok=True)
        print(f"Writing {P_zarr}")
        ds_year.to_zarr(P_zarr, group=f"{current_year}", mode="w")
        yearly_slices = []
        current_year = year
    yearly_slices.append(diff_i)
if yearly_slices:
    ds_year = xr.concat(yearly_slices, dim="time").to_dataset(name="FI_diff")
    P_zarr.parent.mkdir(parents=True, exist_ok=True)
    print(f"Writing {P_zarr}")
    ds_year.to_zarr(P_zarr, group=f"{current_year}", mode="w")

Group '2000' exists in /g/data/gv90/da1339/afim_output/elps-min/zarr/FI_diff.zarr
Group '2000' exists in /g/data/gv90/da1339/afim_output/elps-min/zarr/FI_diff.zarr
Group '2000' exists in /g/data/gv90/da1339/afim_output/elps-min/zarr/FI_diff.zarr
Group '2000' exists in /g/data/gv90/da1339/afim_output/elps-min/zarr/FI_diff.zarr
Group '2000' exists in /g/data/gv90/da1339/afim_output/elps-min/zarr/FI_diff.zarr
Group '2000' exists in /g/data/gv90/da1339/afim_output/elps-min/zarr/FI_diff.zarr
Group '2000' exists in /g/data/gv90/da1339/afim_output/elps-min/zarr/FI_diff.zarr
Group '2000' exists in /g/data/gv90/da1339/afim_output/elps-min/zarr/FI_diff.zarr
Group '2000' exists in /g/data/gv90/da1339/afim_output/elps-min/zarr/FI_diff.zarr
Group '2000' exists in /g/data/gv90/da1339/afim_output/elps-min/zarr/FI_diff.zarr
Group '2000' exists in /g/data/gv90/da1339/afim_output/elps-min/zarr/FI_diff.zarr
Group '2000' exists in /g/data/gv90/da1339/afim_output/elps-min/zarr/FI_diff.zarr
Group '2000' exi

In [7]:
ds = xr.open_zarr("/g/data/gv90/da1339/afim_output/elps-min/zarr/FI_diff.zarr",group="2013",consolidated=False)
ds

<xarray.Dataset> Size: 623MB
Dimensions:  (time: 24, lat: 901, lon: 3601)
Coordinates:
  * lat      (lat) float64 7kB -90.0 -89.9 -89.8 -89.7 ... -0.3 -0.2 -0.1 0.0
  * lon      (lon) float64 29kB -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * time     (time) datetime64[ns] 192B 2013-01-01 2013-01-16 ... 2013-12-12
Data variables:
    FI_diff  (time, lat, lon) float64 623MB dask.array<chunksize=(3, 113, 901), meta=np.ndarray>

##### graphic results

##### compute spatial difference for each time-step (432 in total) and plot for 8 Antarctic regions, show animation

+ [create_FI_diff_plots.py](https://github.com/dpath2o/AFIM/blob/main/scripts/plotting/create_FI_diff_plots.py) is responsible for creating these figures

#### graphic results

In [ ]:
sim_name = "elps-min"
for region in SI_tools.Ant_8sectors.keys():
    SI_tools = SeaIceToolbox(sim_name=sim_name)
    D_png = Path(SI_tools.D_graph, sim_name, region, "FI_diff")
    D_ani = Path(SI_tools.D_graph, "animations", sim_name, "FI_diff")
    F_ani = f"{sim_name}_FI-diff_{region}_2000-2018.mp4"
    P_ani = Path(D_ani,F_ani)
    P_mp4 = Path.home() / "AFIM" / "src" / "AFIM" / "docs" / "figures" / F_ani
    D_ani.mkdir(parents=True, exist_ok=True)
    frames = sorted([f for f in os.listdir(D_png) if f.endswith(".png")])
    os.system(f"rm {SI_tools.D_tmp}/frame_*.png")
    for i, f in enumerate(frames):
        src = D_png / f
        dst = Path(SI_tools.D_tmp) / f"frame_{i:04d}.png"
        if not dst.exists():
            os.symlink(src, dst)
    os.system(f"ffmpeg -y -r 2 -i {SI_tools.D_tmp}/frame_%04d.png -vf \"scale=iw:ih+mod(2-ih\\,2)\" -c:v libx264 -pix_fmt yuv420p {P_ani}")
    os.system(f"cp {P_ani} {P_mp4}")
    

In [ ]:
HTML("""
<video width="1000" controls>
  <source src="https://raw.githubusercontent.com/dpath2o/AFIM/main/docs/figures/elps-min_FI-diff_AS_2000-2018.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

In [ ]:
HTML("""
<video width="1000" controls>
  <source src="https://raw.githubusercontent.com/dpath2o/AFIM/main/docs/figures/elps-min_FI-diff_Aus_2000-2018.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

In [ ]:
HTML("""
<video width="1000" controls>
  <source src="https://raw.githubusercontent.com/dpath2o/AFIM/main/docs/figures/elps-min_FI-diff_BS_2000-2018.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

In [ ]:
HTML("""
<video width="1000" controls>
  <source src="https://raw.githubusercontent.com/dpath2o/AFIM/main/docs/figures/elps-min_FI-diff_DML_2000-2018.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

In [ ]:
HTML("""
<video width="1000" controls>
  <source src="https://raw.githubusercontent.com/dpath2o/AFIM/main/docs/figures/elps-min_FI-diff_EIO_2000-2018.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

In [ ]:
HTML("""
<video width="1000" controls>
  <source src="https://raw.githubusercontent.com/dpath2o/AFIM/main/docs/figures/elps-min_FI-diff_VOL_2000-2018.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

In [ ]:
HTML("""
<video width="1000" controls>
  <source src="https://raw.githubusercontent.com/dpath2o/AFIM/main/docs/figures/elps-min_FI-diff_WIO_2000-2018.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

In [ ]:
HTML("""
<video width="1000" controls>
  <source src="https://raw.githubusercontent.com/dpath2o/AFIM/main/docs/figures/elps-min_FI-diff_WS_2000-2018.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

### Seasonality and inter-annual variability

In [ ]:
sim_name            = "elps-min"
SI_tools            = SeaIceToolbox(sim_name = sim_name)
FI_mets             = xr.open_dataset(Path(SI_tools.D_zarr,"ispd_thresh_5.0e-4","metrics","FI_BT_bool_mets.zarr"))
FIA_obs             = SI_tools.load_AF2020_FI_area_timeseries()
FIA_dict            = {}
FIA_dict['FIA_obs'] = FIA_obs
FIA_dict['FIA_sim'] = FI_mets['FIA']
FIA_obs_df          = FIA_dict['FIA_obs']  # pandas DataFrame
FIA_sim             = FIA_dict['FIA_sim']  # xarray DataArray with .time and daily frequency
obs_daily_list      = []
for year in FIA_obs_df['Year'].unique():
    subset     = FIA_obs_df[FIA_obs_df['Year'] == year]
    doys       = ((subset['DOY_start'] + subset['DOY_end']) // 2).values
    dates      = pd.to_datetime(f"{year}-01-01") + pd.to_timedelta(doys - 1, unit='D')
    ts         = pd.Series(subset['circumpolar'].values, index=dates)
    full_range = pd.date_range(start=f"{year}-01-01", end=f"{year}-12-31", freq='D')
    ts_daily   = ts.reindex(full_range).interpolate("linear").ffill().bfill()
    obs_daily_list.append(ts_daily)
FIA_obs_daily   = pd.concat(obs_daily_list).sort_index()
sim_dates       = pd.to_datetime(FIA_sim['time'].values)
FIA_obs_aligned = FIA_obs_daily.reindex(sim_dates).interpolate("linear").ffill().bfill()
FIA_obs_aligned = FIA_obs_aligned/1e3
FIA_sim_smooth  = FIA_sim.rolling(time=15, center=True).mean()
FIA_diff_norm   = (FIA_obs_aligned - FIA_sim_smooth) / FIA_obs_aligned 
fig, ax1        = plt.subplots(figsize=(20, 12))
ax1.plot(sim_dates, FIA_sim_smooth, label="model (CICE6-standalone)", color='blue', linewidth=1)
ax1.plot(sim_dates, FIA_obs_aligned, label="observed", color='black', linestyle='--')
ax1.tick_params(size=14, axis='y', labelcolor='black')
ax2 = ax1.twinx()
ax2.plot(sim_dates, FIA_diff_norm, label="(Obs - Sim) / Obs [%]", color='gray', alpha=0.5)
ax2.tick_params(size=14, axis='y', labelcolor='gray')
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
plt.title("Circumpolar Antarctic Fast Ice Area 2000-2018: Observations vs Model", fontsize=20)
ax1.set_ylabel("Area (1000 km²)", fontsize=18)
ax2.set_ylabel("Relative Difference (%)", color='gray', fontsize=18)
ax1.set_xlabel("Date", fontsize=18)
ax1.set_ylim([100, 800])
ax2.set_ylim([-1,1])
ax1.set_xlim(pd.Timestamp("2000-03-01"), pd.Timestamp("2018-03-01"))
plt.tight_layout()
plt.show()
results = []
for year in range(2000, 2019):
    t0 = pd.Timestamp(f"{year}-01-01")
    tN = pd.Timestamp(f"{year}-12-31")
    sim_year = FIA_sim_smooth.sel(time=slice(t0, tN)).to_series()
    obs_year = FIA_obs_aligned.loc[t0:tN]
    sim_year = sim_year.dropna()
    obs_year = obs_year.dropna()
    if len(sim_year) == 0 or len(obs_year) == 0:
        continue
    sim_min_val    = sim_year.min()
    sim_min_day    = sim_year.idxmin()
    sim_max_val    = sim_year.max()
    sim_max_day    = sim_year.idxmax()
    obs_min_val    = obs_year.min()
    obs_min_day    = obs_year.idxmin()
    obs_max_val    = obs_year.max()
    obs_max_day    = obs_year.idxmax()
    delta_min_mag  = sim_min_val - obs_min_val
    delta_min_time = (sim_min_day - obs_min_day).days
    delta_max_mag  = sim_max_val - obs_max_val
    delta_max_time = (sim_max_day - obs_max_day).days
    results.append({"year"     : year,
                    "sim_min"  : sim_min_val,
                    "obs_min"  : obs_min_val,
                    "Δmin_mag" : delta_min_mag,
                    "Δmin_day" : delta_min_time,
                    "sim_max"  : sim_max_val,
                    "obs_max"  : obs_max_val,
                    "Δmax_mag" : delta_max_mag,
                    "Δmax_day" : delta_max_time})
comparison_df = pd.DataFrame(results)
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(comparison_df['year'] - 0.2, comparison_df['Δmin_mag'], label='Δmin_mag (Obs - Sim)', color='steelblue')
ax.plot(comparison_df['year'] + 0.2, comparison_df['Δmax_mag'], label='Δmax_mag (Obs - Sim)', color='coral')
ax.set_xlabel("Year")
ax.set_ylabel("Δ Magnitude (km²)")
ax.set_title("Observed - Model Min/Max FIA Magnitude per Year")
ax.legend()
ax.grid(True, axis='y')
plt.tight_layout()
plt.show()
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(comparison_df['year'] - 0.2, comparison_df['Δmin_day'], label='Δmin_day (days)', color='slategray')
ax.plot(comparison_df['year'] + 0.2, comparison_df['Δmax_day'], label='Δmax_day (days)', color='darkorange')
ax.set_xlabel("Year")
ax.set_ylabel("Δ Timing (days)")
ax.set_title("Observed - Model Timing of Min/Max FIA per Year")
ax.axhline(0, color='black', linestyle='--', linewidth=0.8)
ax.legend()
ax.grid(True, axis='y')
plt.tight_layout()
plt.show()


In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/elps-min/AS/FIP_delta/2000-2018_elps-min_AS_FIP_delta.png")

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/elps-min/Aus/FIP_delta/2000-2018_elps-min_Aus_FIP_delta.png")

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/elps-min/BS/FIP_delta/2000-2018_elps-min_BS_FIP_delta.png")

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/elps-min/DML/FIP_delta/2000-2018_elps-min_DML_FIP_delta.png")

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/elps-min/EIO/FIP_delta/2000-2018_elps-min_EIO_FIP_delta.png")

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/elps-min/VOL/FIP_delta/2000-2018_elps-min_VOL_FIP_delta.png")

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/elps-min/WIO/FIP_delta/2000-2018_elps-min_WIO_FIP_delta.png")

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/elps-min/WS/FIP_delta/2000-2018_elps-min_WS_FIP_delta.png")

## Fast ice area and volume comparisons for each tested parameter
1.  What is the optimal concentration of grounded icebergs that should be used?
2.  Is there a relationship between sea ice speed threshold and fast ice area?

In [ ]:
Image(filename="/g/data/gv90/da1339/GRAPHICAL/AFIM/timeseries/FIA_ndte-comparison_1994-1999.png")

### Comparison of internal stress parameter $C^\ast$ 

(``Cstar-min`` : $C^\ast = 10$ and ``Cstar-max`` : $C^\ast = 30$), other parameters $e=2.0$, $k_t=0.2$, $GI-thin=0.25$

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/timeseries/FIA_Cstar_comparison.png")

### Comparison of internal stress parameter $P^\ast$

(``Pstar-min`` : $P^\ast = 5\times10^4$ and ``Pstar-max`` : $P^\ast = 1\times10^4$), other parameters $e=2.0$, $k_t=0.2$, $GI-thin=0.25$

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/timeseries/FIA_Pstar_comparison.png")

### Comparison of tensile stress $k_t$

(``ktens-nil`` : $k_t = 0$, ``ktens-min`` : $k_t = 0.1$, ``ktens-max`` : $k_t = 0.3$, ``ktens-ext`` : $k_t = 0.6$), other parameters $e=2.0$, $GI-thin=0.25$

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/timeseries/FIA_ktens_comparison.png")

### Comparison of ellipse eccentricity

(``elps-max`` : $e = 2.5$, ``elps-mid`` : $e = 1.6$, ``elps-min`` : $e = 1.2$, ``elps-ext`` : $e = 0.8$), other parameters $k_t=0.2$, $GI-thin=0.25$

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/timeseries/FIA_elps_comparison.png")

### Comparison of grounded iceberg concentrations

(``gi-nil`` : $GI_{thin} = 0$, ``gi-min`` : $GI_{thin} = 0.15$, ``gi-mid`` : $GI_{thin} = 0.25$, ``gi-max`` : $GI_{thin} = 0.35$), other parameters $e=2.0$, $k_t=0.2$

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/timeseries/FIA_GI-con_comparison.png")

### Comparison of grounded iceberg ensembles runs

(``elps-min-gi1``, ``elps-min-gi2`` ``elps-min-gi3``), other parameters $e=2.0$, $k_t=0.2$ $GI_{thin}=0.25$

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/timeseries/FIA_GI-var_comparison.png")

### Comparison of sub-cycle iterations

(``ndte``) (``gi-mid`` : ``ndte``$=240$, ``ndte-min`` : ``ndte`` $=120$, ``ndte-max`` : ``ndte``$=720$), other parameters $e=2.0$, $k_t=0.2$, $GI-thin=0.25$

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/timeseries/FIA_ndte_comparison.png")

### Effect of turning off ``revised-EVP``

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/timeseries/FIA_re-EVP_comparison.png")

### The of turning on Rothrock internal ice strength formulation (``kstrength``$=1$) and then tuning Rothrock ice strength parameter $Cf$.

``Roth-cf-def`` $Cf=17$, ``Roth-cf-min`` $Cf=10$, and ``Roth-cf-max`` $Cf=24$

In [ ]:
Image("/g/data/gv90/da1339/GRAPHICAL/AFIM/timeseries/FIA_Roth_comparison.png")

## Do the thermodynamics and mechanical dynamics continue to \`\`behave\'\' when CICE is *heavily tuned* for fast ice?

(limited due to standalone configuration)

## Is there a relationship between sea ice speed threshold and fast ice area?